In [1]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread pandas



In [2]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd
from datetime import datetime, timedelta
import gspread
import re


In [3]:
# Configuration
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly', 'https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = r'C:\Users\avine\Downloads\Project\aristok-automation-441704-40507951b10b.json'  # Path to service account JSON file
SEARCH_CONSOLE_SITE_URL = 'https://www.medikabazaar.com/'  # Replace with your site URL
SPREADSHEET_ID = '1G_xGUUl0yGbtkdza9EKD94dcc3LTKO2ADEOY6tdKwME'  # Replace with your Google Sheet ID

# Authenticate Google APIs
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
search_console_service = build('webmasters', 'v3', credentials=credentials)
sheets_client = gspread.authorize(credentials)


In [4]:
def fetch_monthly_branded_data(year, month, regex_pattern):
    """
    Fetches monthly branded data based on a regex pattern.
    """
    # Calculate start and end dates for the month
    start_date = f"{year}-{month:02d}-01"
    end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).strftime('%Y-%m-%d') if month < 12 else f"{year}-12-31"

    # Fetch data from Google Search Console
    request = {
        'startDate': start_date,
        'endDate': end_date,
        'dimensions': ['query', 'date'],  # Include 'query' for regex matching
    }
    response = search_console_service.searchanalytics().query(siteUrl=SEARCH_CONSOLE_SITE_URL, body=request).execute()
    rows = response.get('rows', [])

    # Filter rows based on the regex pattern for branded queries
    branded_rows = [
        row for row in rows if re.search(regex_pattern, row['keys'][0], re.IGNORECASE)
    ]

    # Summarize data for branded queries
    total_clicks = sum(row.get('clicks', 0) for row in branded_rows)
    total_impressions = sum(row.get('impressions', 0) for row in branded_rows)
    avg_ctr = (sum(row.get('ctr', 0) * row.get('impressions', 0) for row in branded_rows) / total_impressions) if total_impressions else 0
    avg_position = (sum(row.get('position', 0) * row.get('impressions', 0) for row in branded_rows) / total_impressions) if total_impressions else 0

    # Return monthly data
    return {
        'Clicks': total_clicks,
        'Impressions': total_impressions,
        'CTR': f"{avg_ctr * 100:.2f}%",
        'Average Position': f"{avg_position:.1f}" if total_impressions else "-"
    }


In [5]:
def export_to_google_sheets(sheet_id, data):
    """
    Exports a DataFrame to Google Sheets.
    """
    try:
        sheet = sheets_client.open_by_key(sheet_id)
        worksheet = sheet.sheet1  # Use the first sheet

        # Clear existing data and update with new data
        worksheet.clear()
        worksheet.update([data.columns.values.tolist()] + data.values.tolist())
        print("Data successfully exported to Google Sheets.")
    except Exception as e:
        print(f"Error exporting to Google Sheets: {e}")


In [6]:
# Define the year, months, and branded regex pattern
year = 2024
months = ['Jul', 'Aug', 'Sep', 'Oct']  # Customize the months as needed
month_numbers = [7, 8, 9, 10]  # Corresponding numeric months
branded_regex = r'(virtualness|virtual ness|virtualnes|virtuaness|verix)'  # Replace with your custom regex

# Fetch data for each month and store it in a DataFrame
data = []
for month, month_num in zip(months, month_numbers):
    print(f"Fetching data for {month} {year} (branded queries)...")
    monthly_data = fetch_monthly_branded_data(year, month_num, branded_regex)
    data.append({'Performance': month + f"'{str(year)[-2:]}", **monthly_data})

# Convert to DataFrame
df = pd.DataFrame(data)

# Format for Google Sheets
df['Impressions'] = df['Impressions'].apply(lambda x: f"{x:,}")  # Add commas for thousands
print(df)

# Export to Google Sheets
export_to_google_sheets(SPREADSHEET_ID, df)


Fetching data for Jul 2024 (branded queries)...
Fetching data for Aug 2024 (branded queries)...
Fetching data for Sep 2024 (branded queries)...
Fetching data for Oct 2024 (branded queries)...
  Performance  Clicks Impressions    CTR Average Position
0      Jul'24       0           0  0.00%                -
1      Aug'24       0           0  0.00%                -
2      Sep'24       0           0  0.00%                -
3      Oct'24       0           0  0.00%                -
Data successfully exported to Google Sheets.
